# Perplexity Violin Plot Analysis

This notebook reads CSV files from the `ncbi_downloads_sequences_train_40` directory and creates a violin plot showing the distribution of perplexity values across different training steps and a baseline model.

## Data Sources:
- **Step 9**: Early training checkpoint (160 consumed samples)
- **Step 49**: Training checkpoint (800 consumed samples)
- **Step 99**: Training checkpoint (1600 consumed samples)
- **Step 199**: Training checkpoint (3200 consumed samples)
- **Baseline**: nemo2_evo2_7b model

Run the cells below to generate the violin plot.


In [14]:
# Install required packages if not already installed
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import plotly
except ImportError:
    print("Installing plotly...")
    install('plotly')
    
try:
    import pandas
except ImportError:
    print("Installing pandas...")
    install('pandas')

# Note: For saving plots as PDF, you'll also need kaleido:
# pip install kaleido
try:
    import kaleido
except ImportError:
    print("Warning: kaleido not installed. Installing for PDF export...")
    install('kaleido')


In [6]:
# Alternative version with statistics and multiple outputs
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path

# dir_name = "ncbi_downloads_sequences_train_40"
dir_name = "ncbi_downloads_sequences_test_60"
# Define the directory containing CSV files
csv_dir = Path(dir_name)
list_name = dir_name

# Define the CSV files to read with order for display
csv_files = [
    ("Baseline", "ppl_results_merged_nemo2_evo2_7b_8k.csv"),
    ("Step 10", "ppl_results_merged_epoch=0-step=9-consumed_samples=160.csv"),
    ("Step 50", "ppl_results_merged_epoch=0-step=49-consumed_samples=800.csv"),
    ("Step 100", "ppl_results_merged_epoch=0-step=99-consumed_samples=1600.csv"),
    ("Step 200", "ppl_results_merged_epoch=0-step=199-consumed_samples=3200.csv"),
    
]

# Read all CSV files and store data
perplexity_results = []
stats_summary = []

for trace_name, csv_file in csv_files:
    file_path = csv_dir / csv_file
    df = pd.read_csv(file_path)
    perplexity_values = df['perplexity'].values
    
    # Add all perplexity values with their trace name
    for ppl in perplexity_values:
        perplexity_results.append({
            'trace': trace_name,
            'perplexity': ppl
        })
    
    # Calculate statistics
    stats = {
        'Model': trace_name,
        'Count': len(perplexity_values),
        'Mean': np.mean(perplexity_values),
        'Median': np.median(perplexity_values),
        'Std': np.std(perplexity_values),
        'Min': np.min(perplexity_values),
        'Max': np.max(perplexity_values),
        'Q1': np.percentile(perplexity_values, 25),
        'Q3': np.percentile(perplexity_values, 75)
    }
    stats_summary.append(stats)
    
    print(f"Loaded {trace_name}: {len(df)} samples")
    print(f"  - Mean: {stats['Mean']:.4f}, Median: {stats['Median']:.4f}")
    print(f"  - Range: [{stats['Min']:.4f}, {stats['Max']:.4f}]")
    print()

# Convert to DataFrame for plotting
df_plot = pd.DataFrame(perplexity_results)

# Create violin plot using plotly express
fig = px.violin(
    df_plot,
    x="trace",
    y="perplexity",
    box=True,
    points="all",
    labels={"perplexity": "Perplexity", "trace": "Model/Step"},
    title=f"Perplexity Distribution across Training Steps (File: {list_name})",
    category_orders={"trace": [name for name, _ in csv_files]},  # Preserve order
    color="trace",  # Color by trace
    color_discrete_sequence=['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6']
)

# Update layout
fig.update_layout(
    title_x=0.5,
    template="plotly_white",
    width=1200,
    height=700,
    font=dict(size=14),
    showlegend=False
)

# Set y-axis range with some padding for the data
y_min = 0.0
y_max = 6.0
fig.update_yaxes(range=[y_min, y_max])

# Update x-axis
# fig.update_xaxes(tickangle=-30)

fig.update_layout(
    plot_bgcolor='white'
    )
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
)
fig.update_yaxes(
    {'gridcolor': 'lightgrey', 'zerolinecolor': 'lightgrey'},
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    title_standoff=0
)

# Show the plot
fig.show()

# Save outputs
out_file_pdf = f"ppl_violin_plot_{list_name}.pdf"
out_file_html = f"ppl_violin_plot_{list_name}.html"

fig.write_image(out_file_pdf)
fig.write_html(out_file_html)

print(f"\nPlots saved as:")
print(f"  - {out_file_pdf}")
print(f"  - {out_file_html}")



# Display statistics summary
print("\n" + "="*80)
print("Statistical Summary:")
print("="*80)
stats_df = pd.DataFrame(stats_summary)
print(stats_df.to_string(index=False, float_format=lambda x: f'{x:.4f}'))


Loaded Baseline: 1610 samples
  - Mean: 3.6758, Median: 3.6689
  - Range: [1.4462, 4.8776]

Loaded Step 10: 1610 samples
  - Mean: 3.5083, Median: 3.5190
  - Range: [1.6694, 5.0925]

Loaded Step 50: 1610 samples
  - Mean: 2.5120, Median: 2.9613
  - Range: [1.0517, 4.3126]

Loaded Step 100: 1610 samples
  - Mean: 2.1101, Median: 1.5334
  - Range: [1.0343, 4.9593]

Loaded Step 200: 1610 samples
  - Mean: 1.9081, Median: 1.3830
  - Range: [1.0209, 13.8887]




Plots saved as:
  - ppl_violin_plot_ncbi_downloads_sequences_test_60.pdf
  - ppl_violin_plot_ncbi_downloads_sequences_test_60.html

Statistical Summary:
   Model  Count   Mean  Median    Std    Min     Max     Q1     Q3
Baseline   1610 3.6758  3.6689 0.1853 1.4462  4.8776 3.6474 3.7228
 Step 10   1610 3.5083  3.5190 0.2966 1.6694  5.0925 3.4311 3.6684
 Step 50   1610 2.5120  2.9613 1.0730 1.0517  4.3126 1.2263 3.4596
Step 100   1610 2.1101  1.5334 1.0626 1.0343  4.9593 1.1059 3.3106
Step 200   1610 1.9081  1.3830 0.9987 1.0209 13.8887 1.0814 2.5801
